# Trabajo Práctico 1: Análisis Exploratorio de Datos #
## Cientes potenciales a tarjetas de crédito. ##
El [DATASET](https://www.kaggle.com/datasets/sajidhussain3/jobathon-may-2021-credit-card-lead-prediction?select=train.csv) contiene información de clientes de un banco denominado Happy Customer Bank. Es un banco privado de tamaño medio que se ocupa de todo tipo de productos bancarios, como cuentas de ahorro, cuentas corrientes, productos de inversión, productos de crédito, entre otras ofertas.  

En este caso, el Happy Customer Bank desea realizar ventas cruzadas de sus tarjetas de crédito a sus clientes existentes. El banco ha identificado un conjunto de clientes que son elegibles para aceptar estas tarjetas de crédito.

El banco tiene la necesidad, de identificar a los clientes que podrían mostrar intención hacia una tarjeta de crédito recomendada.

<img src = "https://storage.googleapis.com/kaggle-datasets-images/1371711/2277422/2c35585d42b3747882e83e926d0f643e/dataset-cover.png?t=2021-05-27-18-40-42">


 --- 

## 1. Listado de variables y selección ###
### a) Variables de Entrada (vamos a utilizar)##

- **Gender:** Género del cliente. (male/female), no Nans



- **Age:** Edad del cliente. (En años), no NaNs


        
- **Region_Code:** Código de la Región para los clientes. (string), no NaNs. No estan detalladas las regiones por nombre, sin embargo al cliente se le puede demostrar la influencia del dato por medio del código.

        
- **Occupation:** Ocupación del cliente, no NaNs. Valores posibles: 'Entrepreneur', 'Other', 'Salaried','Employed',

        
        
- **Channel_Code:** Código de Canal de Adquisición para el Cliente (Codificado), no NaNs. Valores posibles: X1, X2, X3, X4


        
- **Vintage:** Antiguedad del Cliente (En Meses), no NaNs.

        
        
- **Credit_Product:** Si el Cliente tiene algún producto de crédito activo (Préstamo de vivienda, Préstamo personal, Tarjeta de crédito, etc,algunos Nans. Valor posible: True, False, NaNs.


- **Avg_Account_Balance:** Saldo de cuenta promedio. Saldo promedio de la cuenta del cliente en los últimos 12 meses. Valor posible: Integer, no NaNs

      
      
- **Is_Active:** Si el cliente está activo en los últimos 3 meses. No NaNs
            No-> El cliente no esta activo | Yes-> el cliente esta activo
            
            
--------



### b) Variable de Salida (a predecir) ###

- **Is_Lead:**  Si el Cliente está interesado en la Tarjeta de Crédito.
            0-> El cliente no esta interesado | 1-> el cliente esta interesado

### c) Variable que no vamos a utilizar 

**ID:** Podría conducir a un sobreajuste.

In [ ]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd 
import matplotlib
import keras
import h5py
import PIL
import seaborn as sns
import sklearn
import pytz
import plotly.graph_objects as go

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

pd.options.display.max_columns = 0

plt.rcParams.update({
    "font.family": ["serif"],
    "font.sans-serif": ["Roboto"],
    "font.size": 9,
    "axes.labelsize": 11,
    "axes.titlesize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    'figure.figsize': (15.0, 4.0),
    'axes.grid': False,
    'axes.spines.left': True,
    'axes.spines.right': True,
    'axes.spines.top': True,
    'axes.spines.bottom': True,
})

np.set_printoptions(suppress=True)

warnings.filterwarnings('ignore')

In [ ]:
data_TC = pd.read_csv('train.csv')

BETTER_COLUMN_NAMES = {
    'ID': 'id',
    'Gender': 'sexo',
    'Age': 'edad',
    'Region_Code': 'codigo_region',
    'Occupation': 'ocupacion',
    'Channel_Code': 'codigo_canal',
    'Vintage': 'antiguedad',
    'Credit_Product': 'tiene_producto_credito_activo',
    'Avg_Account_Balance': 'saldo_promedio_cuenta',
    'Is_Active': 'es_activo',
    'Is_Lead': 'esta_interesado',
}
data_TC.rename(columns=BETTER_COLUMN_NAMES, inplace=True)

data_TC.set_index('id', inplace=True)

## 2. Análisis detallado del conjunto de variables ###

### a) Balanceo de variable de Salida ###

In [ ]:
import plotly.express as px 
import numpy 
  
random_x = [100, 2000] 
names = ['Esta interesado', 'No esta interesado'] 
  
fig = px.pie(values=data_TC.esta_interesado.value_counts(), names=names) 
fig.show()

Lo que se puede notar es que la variable se encuentra muy desbalanceada, con un 24% de clientes que no estan interesados en adquirir la tarjeta de crédito.
Esto perjudica el entrenamiento, ya que le dificulta generalizar al modelo. 

### b) Comportamiento variables de entrada ###

### Ocupación ###

In [ ]:
ocupacion=['Entrepreneur', 'Other', 'Salaried','Employed']

fig = go.Figure(data=[
    go.Bar(name='Esta interesado', x=ocupacion, y=data_TC[data_TC.esta_interesado == 0].ocupacion.value_counts()),
    go.Bar(name='No esta interesado', x=ocupacion, y=data_TC[data_TC.esta_interesado == 1].ocupacion.value_counts()),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

Como se puede observar en la gráfica, la mayor cantidad de clientes potenciales tienen como ocupación "Emprendedor".
La dificultad encontrada para el caso de los clientes que son "Empleados" es que la cantidad de casos es muy chica, por lo que se dificulta el dividir los datos y que estos se encuentren representados igualitariamente en Train, Validation y Test.

In [ ]:
genero=['Masculino', 'Femenino']

fig = go.Figure(data=[
    go.Bar(name='Esta interesado', x=genero, y=data_TC[data_TC.esta_interesado == 0].sexo.value_counts()),
    go.Bar(name='No esta interesado', x=genero, y=data_TC[data_TC.esta_interesado == 1].sexo.value_counts()),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [ ]:
df = px.data.tips()
fig = px.histogram(data_TC, x="edad", color="esta_interesado", marginal = 'box', pattern_shape="esta_interesado")
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
df = px.data.tips()
fig = px.histogram(data_TC, x="antiguedad", color="esta_interesado", marginal = 'box')
fig.update_layout(bargap=0.1)
fig.show()

### c) Transformaciones ###

In [ ]:
# 1. Obtener los nombres de las columas como una lista
col_name = data_TC.columns.tolist()

# 2. Itera sobre la lista de nombres
for column in col_name:
    print ("Valores nulos en <{0}>: {1}".format(column, data_TC[column].isnull().sum()))

In [ ]:
Nulos = data_TC.isnull().sum()
print("Columnas             Cantidad de nulos")
print(Nulos)

In [ ]:
data_TC.count()

In [ ]:
#esto capaz nos convenga hacerlo en el mapper con sklearn
data_notNull = data_TC[data_TC['tiene_producto_credito_activo'].notna()]

In [ ]:
Nulos = data_notNull.isnull().sum()
print("Columnas             Cantidad de nulos")
print(Nulos)

In [ ]:
# Se dividen los datos en train y test
# 60% train, 20% test, 20% validation
train, not_train = train_test_split(data_notNull, test_size=0.4)
validation, test = train_test_split(not_train, test_size=0.5)

#Dimención (filas, columnas) de los distintos dataset
print (" Train:", train.shape, '\n', 'Validation:', validation.shape, '\n', 'Test:', test.shape)
#print("first line", "second line", sep="\n")`

In [ ]:
# One Hot Encoder

from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from sklearn.pipeline import Pipeline

# Primer maper

In [ ]:
mapper = DataFrameMapper([
    (['sexo'], [OneHotEncoder()]),
    (['codigo_region'], [OneHotEncoder()]),
    (['ocupacion'], [OneHotEncoder()]),
    (['codigo_canal'], [OneHotEncoder()]),
    (['tiene_producto_credito_activo'], [OneHotEncoder()]),
    (['es_activo'], [OneHotEncoder()])
])

# Lo entrenamos con train
mapper.fit(train)

In [ ]:
# vemos como transforma un sample:
sample = train.sample(5, random_state=20) #Semilla 20

# Sample original:
sample

In [ ]:
# Sample transformado
mapper.transform(sample)

In [ ]:
# Nombres de los faetures
mapper.transformed_names_

In [ ]:
# Descripción de los datos
data_notNull.describe()

### Tipo de cada columna

In [ ]:

# Tipo de cada columna
# ==============================================================================
# En pandas, el tipo "object" hace referencia a strings
# datos.dtypes
data_notNull.info()

In [ ]:
#Primero necesita hacer esto para poder dibujar la matriz de correlaciones
corr_matrix = data_notNull.select_dtypes(include=['float64', 'int']).corr(method='pearson')

In [ ]:
# Heatmap matriz de correlaciones
# ==============================================================================
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10)) # (8, 8) tamaño de la grilla 

sns.heatmap(
    corr_matrix,
    annot     = True,
    cbar      = True, # Barra del costado
    annot_kws = {"size": 20}, # tamaño nro cuadrilla
    vmin      = -1,
    vmax      = 1,
    center    = 0,
    cmap      = sns.diverging_palette(20, 220, n=200), # (20, 220, n=200) color de la grilla
    square    = False, 
    ax        = ax
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation = 45,
    horizontalalignment = 'right',
)

ax.tick_params(labelsize = 10) # (labelsize = 50) tamaño la leyenda de los ejes

- **Gender:** one-hot encoder



- **Age:** rango


        
- **Region_Code:** one-hot encoder


        
- **Occupation:** one-hot encoder

        
        
- **Channel_Code:** one-hot encoder


        
- **Vintage:** rango

        
        
- **Credit_Product:** one-hot encoder y eliminamos los null. (esta desbalanceado)



- **Avg_Account_Balance:** escalar y rangos (y ver outliers)

      
      
- **Is_Active:** one-hot encoder